In [ ]:
from collections import deque 

def slidingPuzzle(start_state):
    # Convert input list to string representation
    target = "01234"
    start = ''.join(map(str, start_state))
    
    if start == target:
        print("Already at goal!")
        return 0
        
    # valid moves for each position (1D board with 5 positions)
    adj = {
        0: [1, 3],
        1: [0, 2, 4],
        2: [1],
        3: [0, 4],
        4: [1, 3],
    }
    
    # to track which state you've explored
    visited = set()
    visited.add(start)
    
    # Queue holds (current_state_string, index_of_blank, moves)
    q = deque([(start, start.index('0'), 0)])
    
    while q:
        state, pos0, moves = q.popleft()
        
        if state == target:
            print(f"Number of moves: {moves}")
            return moves
            
        for neighbor in adj[pos0]:
            new_state = list(state)
            new_state[pos0], new_state[neighbor] = new_state[neighbor], new_state[pos0]
            s_str = ''.join(new_state)
            
            if s_str not in visited:
                visited.add(s_str)
                q.append((s_str, neighbor, moves + 1))
                
    print("Unsolvable puzzle.")
    return -1

# Enhanced test cases with expected results
tests = [
    ([0, 1, 2, 3, 4], "Already solved", 0),
    ([1, 0, 2, 3, 4], "One move away", 1),
    ([1, 2, 0, 3, 4], "Two moves away", 2),
    ([2, 1, 0, 3, 4], "Three moves away", 3),
    ([3, 1, 2, 4, 0], "Scrambled but solvable", 5),
    ([1, 2, 3, 0, 4], "Harder scramble", 4),
    ([4, 3, 2, 1, 0], "Worst case (far from goal)", 8),
]

print("\n===== SLIDING PUZZLE SOLUTION TESTER =====")
for board, description, expected in tests:
    print(f"\n🔷 Test: {board} — {description}")
    print(f"Expected moves: {expected}")
    result = slidingPuzzle(board)
    if result == expected:
        print(f"✅ PASS: Got {result} moves as expected")
    else:
        print(f"❌ FAIL: Expected {expected} moves, got {result}")

print("\n===== TESTING COMPLETE =====")